In [1]:
import pandas as pd
import math
import numpy as np
import pyodbc
from sqlalchemy import create_engine
import urllib

In [2]:
# level_col = 0
# item_col = 0
# desc_col = 1
# unit_col = 3
# total_col = 4

In [20]:
item_col = 0
desc_col = 1
quant_col = 2
unit_col = 3
rate_col = 4
amount_col = 5


In [3]:
params = urllib.parse.quote_plus('DRIVER={SQL Server};SERVER=WINDOWS-UJU3M60\SQLEXPRESS;DATABASE=SMEC;UID=SMEC;PWD=smecs')
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [4]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=WINDOWS-UJU3M60\SQLEXPRESS;DATABASE=SMEC;UID=SMEC;PWD=smecs')
cursor = cnxn.cursor()

cursor.execute("SELECT * FROM Form_IDs")
for row in cursor.fetchall():
    print (row)

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Login failed for user 'SMEC'.  Reason: The password of the account has expired. (18487) (SQLDriverConnect); [42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Login failed for user 'SMEC'.  Reason: The password of the account has expired. (18487)")

In [21]:
#Read in Excel File to get sheet names
#excel = pd.ExcelFile("Tender Schedule (NEW Branding).xlsx")
excel = pd.ExcelFile(r"C:\Users\User\Documents\UNI\Project\Python\master schedule 221014 numbered.xlsx")
sheets = excel.sheet_names

In [22]:
df_forms = pd.DataFrame(columns = {"Form ID", "Form Name"})

In [23]:
#Read pages and convert to Data Frames
#Data Frames stored in DF_List
df_list = list()
inc = 0
for x in sheets :
    df_list.append(pd.read_excel(r"C:\Users\User\Documents\UNI\Project\Python\master schedule 221014 numbered.xlsx", sheet_name=x))
    form = [[inc,x]]
    df_forms = df_forms.append(pd.DataFrame(form,columns = ["Form ID", "Form Name"]),ignore_index = True)
    inc += 1

C:\Users\User\AppData\Local\Temp\ipykernel_17740\3483317805.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_forms = df_forms.append(pd.DataFrame(form,columns = ["Form ID", "Form Name"]),ignore_index = True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\3483317805.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_forms = df_forms.append(pd.DataFrame(form,columns = ["Form ID", "Form Name"]),ignore_index = True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\3483317805.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_forms = df_forms.append(pd.DataFrame(form,columns = ["Form ID", "Form Name"]),ignore_index = True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\3483317805.py:8: FutureWarning: The frame.

In [29]:
print(df_list)

[                          Unnamed: 0  \
0                       Contract No:   
1                           Project:   
2                         Principal:   
3                        Contractor:   
4                                NaN   
5   CONSTRUCTION OF BULK  EARTHWORKS   
6                   PRICING SCHEDULE   
7                                NaN   
8                               Item   
9                                  1   
10                               1.1   
11                               1.2   
12                               1.3   
13                               1.4   
14                               1.5   
15                               1.6   
16                               1.7   
17                               1.8   
18                               1.9   
19                              1.10   
20                              1.11   
21                              1.12   
22                              1.13   
23                              1.14   

In [25]:
df_col = ["Form ID", "Level 1", "Level 2", "Level 3", "Level 4", "Level 5", "Item ID","Description","Units"]
df = pd.DataFrame(columns = df_col)
print(df)

Empty DataFrame
Columns: [Form ID, Level 1, Level 2, Level 3, Level 4, Level 5, Item ID, Description, Units]
Index: []


In [26]:
for x in range(0,len(sheets)-1):
    inside = False
    for index, rows in df_list[x].iterrows():
        if inside :
            #Check if at end of table
            if rows[item_col] == "" :
                inside = False
            else :
                hold = ""
                df_hold = ["","0","0","0","0","0","","",""]
                full_count = 0
                rows[item_col] = str(rows[item_col])
                if rows[item_col] != "nan":
                    count = 0
                    length = len(rows[item_col])
                    for i in str(rows[item_col]):
                        count += 1
                        if i != ".":
                            hold = hold + i
                            if count == length:
                                df_hold[full_count + 1] = hold
                        else:
                            df_hold[full_count + 1] = hold
                            full_count += 1
                            hold = ""
                    df_hold[7] = rows[desc_col]
                    df_hold[0] = df_forms['Form ID'][x]
                    for y in range(0,6):
                        df_hold[6] = str(df_hold[6]) + str(df_hold[y]) + "."
                    df_hold[6] = df_hold[6][:-1]
                    df_hold[8] = rows[unit_col]
                    #PUSH TO SQL HERE
                    s_row = pd.Series(df_hold, index=df.columns)
                    df = df.append(s_row,ignore_index=True)
                    print(df_hold)       

        if rows[item_col] == "Item" :
            inside = True

C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp

[0, '1', '0', '0', '0', '0', '0.1.0.0.0.0', 'EARTHWORKS', nan]
[0, '1', '1', '0', '0', '0', '0.1.1.0.0.0', 'Stripping', 'sq.m']
[0, '1', '2', '0', '0', '0', '0.1.2.0.0.0', 'Cut material ', 'cu.m']
[0, '1', '3', '0', '0', '0', '0.1.3.0.0.0', 'Importation of approved material, Excl S&C', 'cu.m']
[0, '1', '4', '0', '0', '0', '0.1.4.0.0.0', 'Spread and compact fill - from CUT Areas', 'cu.m']
[0, '1', '5', '0', '0', '0', '0.1.5.0.0.0', 'Spread and compact fill - from TEMP OUTFALL DRAINS', 'cu.m']
[0, '1', '6', '0', '0', '0', '0.1.6.0.0.0', 'Spread and compact fill - from Wetlands', 'cu.m']
[0, '1', '7', '0', '0', '0', '0.1.7.0.0.0', 'Spread and compact fill - Imported', 'cu.m']
[0, '1', '8', '0', '0', '0', '0.1.8.0.0.0', 'Disposal of excess material offsite (unsuitable)', 'cu.m']
[0, '1', '9', '0', '0', '0', '0.1.9.0.0.0', 'Importation of approved material', 'cu.m']
[0, '1', '10', '0', '0', '0', '0.1.10.0.0.0', 'Stockpile for future use', 'cu.m']
[0, '1', '11', '0', '0', '0', '0.1.11.0.0.0'

C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp

[1, '4', '1', '1', '10', '0', '1.4.1.1.10.0', '450mm Dia. 1.5-2.5m deep', 'lin.m']
[1, '4', '1', '1', '11', '0', '1.4.1.1.11.0', '450mm Dia. 2.5-3.5m deep', 'lin.m']
[1, '4', '1', '1', '12', '0', '1.4.1.1.12.0', '525mm Dia. 1.5-2.5m deep', 'lin.m']
[1, '4', '1', '1', '13', '0', '1.4.1.1.13.0', '525mm Dia. 2.5-3.5m deep', 'lin.m']
[1, '4', '1', '1', '14', '0', '1.4.1.1.14.0', '600mm Dia. 1.5-2.5m deep', 'lin.m']
[1, '4', '1', '1', '15', '0', '1.4.1.1.15.0', '600mm Dia. 2.5-3.5m deep', 'lin.m']
[1, '4', '1', '1', '16', '0', '1.4.1.1.16.0', '675mm Dia. 1.5-2.5m deep', 'lin.m']
[1, '4', '1', '1', '17', '0', '1.4.1.1.17.0', '675mm Dia. 2.5-3.5m deep', 'lin.m']
[1, '4', '1', '1', '18', '0', '1.4.1.1.18.0', '750mm Dia. 1.5-2.5m deep', 'lin.m']
[1, '4', '1', '1', '19', '0', '1.4.1.1.19.0', '750mm Dia. 2.5-3.5m deep', 'lin.m']
[1, '4', '1', '2', '0', '0', '1.4.1.2.0.0', 'Extra over item 4.1.1 for CR backfill', nan]
[1, '4', '1', '2', '1', '0', '1.4.1.2.1.0', '225mm Dia. <1.5m deep', 'lin.m']
[1

C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp

...

C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp

[4, '1', '2', '1', '0', '0', '4.1.2.1.0.0', '150 Ø - F Backfill <1.5m', 'lin.m']
[4, '1', '2', '2', '0', '0', '4.1.2.2.0.0', '150 Ø - F Backfill 1.5-2.5m', 'lin.m']
[4, '1', '2', '3', '0', '0', '4.1.2.3.0.0', '150 Ø - F Backfill 2.5-3.5m', 'lin.m']
[4, '1', '2', '4', '0', '0', '4.1.2.4.0.0', '150 Ø - F Backfill 3.5-4.5m', 'lin.m']
[4, '1', '2', '5', '0', '0', '4.1.2.5.0.0', '150 Ø - F Backfill 4.5-5.5m', 'lin.m']
[4, '1', '2', '6', '0', '0', '4.1.2.6.0.0', '150 Ø - F Backfill 5.5-6.5m', 'lin.m']
[4, '1', '2', '7', '0', '0', '4.1.2.7.0.0', '225 Ø - F Backfill <1.5m', 'lin.m']
[4, '1', '2', '8', '0', '0', '4.1.2.8.0.0', '225 Ø - F Backfill 1.5-2.5m', 'lin.m']
[4, '1', '2', '9', '0', '0', '4.1.2.9.0.0', '225 Ø - F Backfill 2.5-3.5m', 'lin.m']
[4, '1', '2', '10', '0', '0', '4.1.2.10.0.0', '225 Ø - F Backfill 3.5-4.5m', 'lin.m']
[4, '1', '2', '11', '0', '0', '4.1.2.11.0.0', '225 Ø - F Backfill 4.5-5.5m', 'lin.m']
[4, '1', '2', '12', '0', '0', '4.1.2.12.0.0', '225 Ø - F Backfill 5.5-6.5m', '

C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp

[4, '2', '3', '0', '0', '0', '4.2.3.0.0.0', 'Maintenance holes - 1500mm dia', nan]
[4, '2', '3', '1', '0', '0', '4.2.3.1.0.0', 'MH1500 - 1.5-2.5m deep', 'No.']
[4, '2', '3', '2', '0', '0', '4.2.3.2.0.0', 'MH1500 - 2.5-3.5m deep', 'No.']
[4, '2', '3', '3', '0', '0', '4.2.3.3.0.0', 'MH1500 - 3.5-4.5m deep', 'No.']
[4, '2', '3', '4', '0', '0', '4.2.3.4.0.0', 'MH1500 - 4.5-5.5m deep', 'No.']
[4, '2', '3', '5', '0', '0', '4.2.3.5.0.0', 'MH1500 - 5.5-6.5m deep', 'No.']
[4, '2', '4', '0', '0', '0', '4.2.4.0.0.0', 'Maintenance holes - 1800mm dia', nan]
[4, '2', '4', '1', '0', '0', '4.2.4.1.0.0', 'MH1800 - 1.5-2.5m deep', 'No.']
[4, '2', '4', '2', '0', '0', '4.2.4.2.0.0', 'MH1800 - 2.5-3.5m deep', 'No.']
[4, '2', '4', '3', '0', '0', '4.2.4.3.0.0', 'MH1800 - 3.5-4.5m deep', 'No.']
[4, '2', '4', '4', '0', '0', '4.2.4.4.0.0', 'MH1800 - 4.5-5.5m deep', 'No.']
[4, '2', '4', '5', '0', '0', '4.2.4.5.0.0', 'MH1800 - 5.5-6.5m deep', 'No.']
[4, '2', '5', '0', '0', '0', '4.2.5.0.0.0', 'Maintenance chamber

C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_17740\1677801870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(s_row,ignore_index=True)


In [18]:
df = df.reset_index(drop=True)

In [36]:
df.to_sql('Tender Master', engine, if_exists='replace')

In [37]:
df_forms.to_sql('Form_IDs', engine, if_exists='replace')

In [28]:
df.to_csv("test.csv")

In [10]:
df_forms.to_csv("Form_IDs.csv")